In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # 将SettingWithCopyWarning隐藏
import numpy as np
import os,math,time,re
from pathlib import Path
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()#归一化
shijian = time.strftime('%Y-%m-%d',time.localtime(time.time()))

```python
步骤：
    1. 读取数据
    2. 处理数据
        2.1 过滤未开通的线路站点，只统计已开通的线路，因为未开通的线路对站点换乘的数量及站台数量都可能产生影响
        2.2 线路名称处理，去除外圈，内圈等影响线路数量判断的词
        2.3 站台数量处理，统计相同城市相同站点名称但经纬度不同的数量作为站台数量值
        2.4 站点换乘线路数量统计，统计经过线路的名称唯一值。处理线路分叉(Y叉及X叉)，环线
        2.5 站点分数赋值
    3. 数据指标归一（每个站点的归一化操作都以站点所在的线路作为基准）
        3.1 站点所在位置的分数归一化（中间站分数为线路的站点数量，往两边递减）
        3.2 站点可换成线路数量的归一化
        3.3 站点包含站台数量的归一化
    4. 站点去重--根据线路将站点归一化后的分数进行相加，得到最终的站点分数
    5. 权重比测试
    6. 结果验证
```

In [2]:
#1. 读取数据
df = pd.read_csv('./subway_station_raw_2023-06-01.csv')
df.fillna('',inplace=True)

In [3]:
#2 处理数据
#2.1 过滤未开通的线路站点
data = df[df['line_open_status']==1]
print('全量地铁站数量：',len(df))
print('已开通地铁站数量：',len(data))
#未进行地铁站去重处理，每个方向的地铁线路的站点都会计算一次，后面的数据流程会处理

全量地铁站数量： 9629
已开通地铁站数量： 9003


In [4]:
#2.2 线路名称处理，去除影响判断的特殊词
#2.3 站台数量统计,通过poiid进行分组，并统计pointx或pointy的唯一值数量【因为站点的位置决定其是否为相同站台】:platform_count_final


platforms = data.groupby('poiid')['pointx'].nunique()#2.3

for index,row in data.iterrows():
    name_raw = row['line_name']
    pattern = r'\(.*?\)|（.*?）'
    line_name_clean = re.sub(pattern, '', name_raw)
    data.loc[index,'platform_count_final'] = platforms[row['poiid']]
    data.loc[index,'line_name_clean'] = line_name_clean.replace('外环','').replace('内环','').replace('外圈','').replace('内圈','')



In [5]:
#2.4 站点可换成线路数量统计:trans_count_final
trans_lines = pd.DataFrame(data.groupby(['poiid'])['line_name_clean'].nunique())#2.4
trans_lines.rename(columns={'line_name_clean':'trans_count_final'},inplace=True)
data = pd.merge(data,trans_lines,on='poiid')

In [6]:
#2.5 计算每条线路的中间站点分值,全线路需要进行归一化处理，所以可以保留线路站点数量值作为最大值
line_score = {}#存储每条线路的最大值
middle_index = {}#存储每条线路的中间站点index
for line_uid in data['line_uid'].unique():
    line_data = data[data['line_uid'] == line_uid]#获取每条线路的数据表
    line_length = len(line_data)
    max_score = line_length
    midd_index = math.ceil(line_length/2)
    line_score[line_uid] = max_score
    middle_index[line_uid] = midd_index

In [7]:
#2.5 给每个站点赋值所在位置的分数:station_score
for index, row in data.iterrows():
    #获取线路最大评分及中间站索引
    line_max_score = int(line_score[row['line_uid']])
    mid_index = int(middle_index[row['line_uid']])

    if  row['stations_count'] % 2 ==0:#判断中间站为偶数，表示线路的站点数量为偶数
        if row['station_index']<=mid_index:#站点位于中间站左侧
            station_score = line_max_score-(mid_index-row['station_index'])
        elif row['station_index']==mid_index +1:
            station_score = line_max_score
        else:
            station_score = line_max_score -abs(row['station_index']-mid_index-1)
    else:#判断中间站为基数,表示线路的站点数量为奇数
        if row['station_index']<=mid_index:#站点位于中间站左侧
            station_score = line_max_score-(mid_index-row['station_index'])
        else:
            station_score = line_max_score - abs(row['station_index']-mid_index)
    data.loc[index,'station_score'] = station_score
    # data.loc[index,'mid_index'] = mid_index
    # data.loc[index,'line_max_score'] = line_max_score

In [8]:
#3. 数据指标归一化
#3.1 将每条单独线路的站点分数归一化
station_score_normalized = pd.DataFrame()
for line_uid in data['line_uid'].unique():
    line_data = data[data['line_uid'] == line_uid]#获取每条线路的数据表
    line_data['station_score_normalized'] = scaler.fit_transform(line_data[['station_score']])
    station_score_normalized = pd.concat([station_score_normalized,line_data])

In [9]:
#3.1.1 合并表格，根据列合并，保留station_score_normalized字段
result = pd.merge(data[data.columns], station_score_normalized[['station_uid', 'station_score_normalized']], on='station_uid')
#3.2 站点可换成线路数量的归一化
#3.3 站点包含站台数量的归一化
result[['trans_lines_normalized',  'platform_normalized']] = scaler.fit_transform(result[['trans_count_final', 'platform_count_final']])


In [70]:
#4. 站点去重--根据线路将站点归一化后的分数进行相加，得到最终的站点分数
#4.1 根据线路名称，站点名称，城市code进行去重（目的是为了处理一个站点在双向上的重复计算，后续需要优化相同线路上不同站点分数的处理，'上海市14号线曹杨路站'）
sub_dup = result.drop_duplicates(subset=['line_name_clean','station_name','ccode'])

#4.2 根据站点的统一ID，站点名称，城市code进行分类，对站点评分，换乘数量评分，站台数量评分跟别进行求和，求最大值，求最大值操作，主要是为了解决相同站点在不同线路评分
station_com= sub_dup.groupby(['poiid','station_name','ccode']).agg({'station_score_normalized':np.sum,'trans_lines_normalized':np.max,'platform_normalized':np.max})
station_com.rename(columns={'station_score_normalized':'station_score_normalized_final','trans_lines_normalized':'trans_lines_normalized_final','platform_normalized':'platform_normalized_final'},inplace=True)
#4.3 将分类统计后的结果与sub_dup进行merge，保留poiid，站点名称，经度，纬度，站点分数，换乘分数，站台分数，最终总分
station_evaluation = pd.merge(sub_dup,station_com,on='poiid')
station_evaluation.drop_duplicates(subset='poiid',inplace=True)


In [74]:
#3. 关键指标归一化处理
#4. 站台数量&换乘线路数量可以在全局范围进行归一化，因为这两个因素主要与站点本身相关度更高，与所在线路因素关联不大

#V1权重：换乘线路数量0.4，站台数量0.2，站台评分0.3，线路长度0.1--效果不行
#V2权重：换乘线路数量0.3，站台数量0.1，站台评分0.5，线路长度0.1--效果不行
#V3修改位置:
    #1. 去除线路长度stations_count_normalized入参，因为站点的分数已经受线路长度影响，已经在站点的分数里面直接体现了
    #2. 站点分数逻辑修改：每条线路根据站点数量归一化后在赋值给每个站点分值
#V3权重：换乘线路数量0.3，站台数量0.2，站台评分0.5
#V4权重：换乘线路数量0.6，站台数量0.2，站台评分0.2

station_evaluation['final_score'] = station_evaluation[['trans_lines_normalized_final', 'platform_normalized_final', 'station_score_normalized_final']].apply(
    lambda row: row['trans_lines_normalized_final']*0.6+row['platform_normalized_final']*0.2+row['station_score_normalized_final']*0.2, axis=1)


In [31]:
station_evaluation.to_csv('./station_evaluation{0}.csv'.format(shijian))

In [67]:
# result.to_excel("D:/Desktop/station20230601.xlsx")